In [1]:
import sys
import os
import re
import pandas as pd
import numpy as np

In [3]:
county_facts = pd.read_csv('county_facts.csv')

In [29]:
county_facts.set_index(county_facts['fips'])
county_facts

fips           area_name state_abbreviation  PST045214  PST040210  \
0         0       United States                NaN  318857056  308758105   
1      1000             Alabama                NaN    4849377    4780127   
2      1001      Autauga County                 AL      55395      54571   
3      1003      Baldwin County                 AL     200111     182265   
4      1005      Barbour County                 AL      26887      27457   
5      1007         Bibb County                 AL      22506      22919   
6      1009       Blount County                 AL      57719      57322   
7      1011      Bullock County                 AL      10764      10915   
8      1013       Butler County                 AL      20296      20946   
9      1015      Calhoun County                 AL     115916     118586   
10     1017     Chambers County                 AL      34076      34170   
11     1019     Cherokee County                 AL      26037      25986   
12     1021      Chilton County                 AL      43931      43631   
13     1023      Choctaw County                 AL      13323      13858   
14     1025       Clarke County                 AL      24945      25840   
15     1027         Clay County                 AL      13552      13932   
16     1029     Cleburne County                 AL      15080      14972   
17     1031       Coffee County                 AL      50909      49948   
18     1033      Colbert County                 AL      54543      54428   
19     1035      Conecuh County                 AL      12670      13228   
20     1037        Coosa County                 AL      10886      11758   
21     1039    Covington County                 AL      37914      37765   
22     1041     Crenshaw County                 AL      13977      13906   
23     1043      Cullman County                 AL      81289      80410   
24     1045         Dale County                 AL      49484      50251   
25     1047       Dallas County                 AL      41711      43820   
26     1049       DeKalb County                 AL      71065      71115   
27     1051       Elmore County                 AL      80977      79296   
28     1053     Escambia County                 AL      37733      38319   
29     1055       Etowah County                 AL     103531     104427   
...     ...                 ...                ...        ...        ...   
3165  55131   Washington County                 WI     133251     131885   
3166  55133     Waukesha County                 WI     395118     389938   
3167  55135      Waupaca County                 WI      52066      52410   
3168  55137     Waushara County                 WI      24178      24496   
3169  55139    Winnebago County                 WI     169511     166994   
3170  55141         Wood County                 WI      73608      74749   
3171  56000             Wyoming                NaN     584153     563767   
3172  56001       Albany County                 WY      37811      36299   
3173  56003     Big Horn County                 WY      11930      11668   
3174  56005     Campbell County                 WY      48320      46133   
3175  56007       Carbon County                 WY      15854      15885   
3176  56009     Converse County                 WY      14097      13833   
3177  56011        Crook County                 WY       7248       7083   
3178  56013      Fremont County                 WY      40703      40123   
3179  56015       Goshen County                 WY      13514      13247   
3180  56017  Hot Springs County                 WY       4816       4812   
3181  56019      Johnson County                 WY       8573       8569   
3182  56021      Laramie County                 WY      96389      91881   
3183  56023      Lincoln County                 WY      18567      18106   
3184  56025      Natrona County                 WY      81624      75450   
3185  56027     Niobrara County                 WY     

The cell below prints out the definitions matching each county fact abbreviation.

In [26]:
county_facts_dictionary = open('county_facts_dictionary.csv', 'r')
lines = county_facts_dictionary.readlines()
for line in lines:
    print(line)

column_name,description

PST045214,"Population, 2014 estimate"

PST040210,"Population, 2010 (April 1) estimates base"

PST120214,"Population, percent change - April 1, 2010 to July 1, 2014"

POP010210,"Population, 2010"

AGE135214,"Persons under 5 years, percent, 2014"

AGE295214,"Persons under 18 years, percent, 2014"

AGE775214,"Persons 65 years and over, percent, 2014"

SEX255214,"Female persons, percent, 2014"

RHI125214,"White alone, percent, 2014"

RHI225214,"Black or African American alone, percent, 2014"

RHI325214,"American Indian and Alaska Native alone, percent, 2014"

RHI425214,"Asian alone, percent, 2014"

RHI525214,"Native Hawaiian and Other Pacific Islander alone, percent, 2014"

RHI625214,"Two or More Races, percent, 2014"

RHI725214,"Hispanic or Latino, percent, 2014"

RHI825214,"White alone, not Hispanic or Latino, percent, 2014"

POP715213,"Living in same house 1 year & over, percent, 2009-2013"

POP645213,"Foreign born persons, percent, 2009-2013"

POP815213,"Langua

Our country fact data are on different scales and could cause us to reach a poor fit for the models we will train. Since we cannot assume a priori that all the features are normally distributed we will instead scale each feature using: $x\rightarrow\frac{x-\min{x}}{\max{x}-\min{x}}$

In [27]:
cols = county_facts.columns.values
print(cols)

['fips' 'area_name' 'state_abbreviation' 'PST045214' 'PST040210'
 'PST120214' 'POP010210' 'AGE135214' 'AGE295214' 'AGE775214' 'SEX255214'
 'RHI125214' 'RHI225214' 'RHI325214' 'RHI425214' 'RHI525214' 'RHI625214'
 'RHI725214' 'RHI825214' 'POP715213' 'POP645213' 'POP815213' 'EDU635213'
 'EDU685213' 'VET605213' 'LFE305213' 'HSG010214' 'HSG445213' 'HSG096213'
 'HSG495213' 'HSD410213' 'HSD310213' 'INC910213' 'INC110213' 'PVY020213'
 'BZA010213' 'BZA110213' 'BZA115213' 'NES010213' 'SBO001207' 'SBO315207'
 'SBO115207' 'SBO215207' 'SBO515207' 'SBO415207' 'SBO015207' 'MAN450207'
 'WTN220207' 'RTN130207' 'RTN131207' 'AFN120207' 'BPS030214' 'LND110210'
 'POP060210']


We can ignore the first 3 columns, and only look at the first ten features for now.

In [28]:
cols = cols[3:13]
cols

array(['PST045214', 'PST040210', 'PST120214', 'POP010210', 'AGE135214',
       'AGE295214', 'AGE775214', 'SEX255214', 'RHI125214', 'RHI225214'], dtype=object)

In [24]:
cf = county_facts[cols].copy() # A partial copy of county facts for us to modify.
for c in cols:
    c_max = county_facts[c].max()
    c_min = county_facts[c].min()
    #print(c_max)
    #print(c_min)
    cf[c] = (cf[c] - c_min)/(c_max - c_min)
cf

POP010210  AGE135214  AGE295214  AGE775214  SEX255214  RHI125214  \
0      1.000000   0.452555   0.550000   0.274102   0.894366   0.779456   
1      0.015481   0.445255   0.542857   0.289225   0.906690   0.701913   
2      0.000176   0.437956   0.600000   0.260870   0.904930   0.784491   
3      0.000590   0.408759   0.528571   0.353497   0.901408   0.877140   
4      0.000089   0.416058   0.504762   0.311909   0.820423   0.505539   
5      0.000074   0.386861   0.500000   0.279773   0.808099   0.768379   
6      0.000185   0.445255   0.561905   0.321361   0.889085   0.966767   
7      0.000035   0.459854   0.509524   0.281664   0.797535   0.270896   
8      0.000068   0.445255   0.561905   0.340265   0.943662   0.542800   
9      0.000384   0.416058   0.528571   0.302457   0.911972   0.763343   
10     0.000111   0.430657   0.509524   0.345936   0.920775   0.587110   
11     0.000084   0.350365   0.485714   0.395085   0.883803   0.936556   
12     0.000141   0.467153   0.576190   0.287335   0.894366   0.877140   
13     0.000045   0.357664   0.490476   0.393195   0.924296   0.569990   
14     0.000083   0.408759   0.538095   0.340265   0.929577   0.543807   
15     0.000045   0.386861   0.514286   0.366730   0.890845   0.833837   
16     0.000048   0.416058   0.554762   0.347826   0.887324   0.951662   
17     0.000162   0.445255   0.564286   0.300567   0.889085   0.769386   
18     0.000176   0.416058   0.516667   0.349716   0.911972   0.813696   
19     0.000043   0.401460   0.509524   0.385633   0.913732   0.524673   
20     0.000037   0.350365   0.435714   0.349716   0.871479   0.665660   
21     0.000122   0.430657   0.526190   0.374291   0.908451   0.848943   
22     0.000045   0.445255   0.540476   0.340265   0.899648   0.725076   
23     0.000260   0.430657   0.535714   0.332703   0.889085   0.968781   
24     0.000162   0.459854   0.561905   0.291115   0.889085   0.757301   
25     0.000142   0.489051   0.600000   0.294896   0.945423   0.292044   
26     0.000230   0.452555   0.588095   0.296786   0.892606   0.930514   
27     0.000257   0.430657   0.547619   0.264650   0.906690   0.765358   
28     0.000124   0.445255   0.523810   0.315690   0.859155   0.626385   
29     0.000338   0.416058   0.526190   0.327032   0.908451   0.817724   
...         ...        ...        ...        ...        ...        ...   
3165   0.000427   0.386861   0.547619   0.296786   0.885563   0.967774   
3166   0.001263   0.372263   0.533333   0.311909   0.896127   0.942598   
3167   0.000169   0.379562   0.504762   0.368620   0.875000   0.980866   
3168   0.000079   0.350365   0.445238   0.425331   0.832746   0.961732   
3169   0.000541   0.408759   0.495238   0.277883   0.875000   0.936556   
3170   0.000242   0.416058   0.521429   0.351607   0.890845   0.960725   
3171   0.001825   0.474453   0.564286   0.264650   0.862676   0.933535   
3172   0.000117   0.379562   0.392857   0.185255   0.839789   0.922457   
3173   0.000038   0.474453   0.602381   0.362949   0.875000   0.965760   
3174   0.000149   0.583942   0.666667   0.134216   0.845070   0.956697   
3175   0.000051   0.452555   0.569048   0.260870   0.802817   0.947633   
3176   0.000045   0.503650   0.588095   0.266541   0.862676   0.962739   
3177   0.000023   0.481752   0.559524   0.340265   0.871479   0.969789   
3178   0.000130   0.547445   0.607143   0.306238   0.880282   0.752266   
3179   0.000043   0.408759   0.478571   0.387524   0.843310   0.965760   
3180   0.000015   0.321168   0.495238   0.446125   0.885563   0.961732   
3181   0.000027   0.423358   0.523810   0.389414   0.869718   0.960725   
3182   0.000297   0.474453   0.559524   0.270321   0.873239   0.919436   
3183   0.000058   0.481752   0.642857   0.279773   0.867958   0.972810   
3184   0.000244   0.496350   0.569048   0.241966   0.871479   0.944612   
3185   0.000008   0.343066   0.395238   0.400756   0.955986   0.956697   
3186   0.000091   0.401460   0.488095   0.379962   0.882042   0.962739   
3187   0.0